In [5]:
from pyDOE import *
thing = lhs(2, samples=3, criterion='corr')

new candidate solution found with max,abs corrcoef = 1.0


In [6]:
thing * 100

array([[22.83547676, 47.50107142],
       [77.2625648 , 91.76499667],
       [34.62375317, 31.44456339]])